In [0]:
%sql
USE CATALOG agriculture

In [0]:
from pyspark.sql.functions import col, year, avg, max, stddev, round, expr, when, lower, trim

# 1. Read YOUR existing Silver Table
df_silver_market = spark.read.table("agriculture.silver.market_prices")

# 2. Filter for Recent Data
df_recent = df_silver_market.filter(year(col("market_date")) >= 2020)

# --- THE FIX STARTS HERE ---
# 3. Normalize State Names to match Gov Yield Data
df_normalized = df_recent.withColumn("state_clean", 
    when(col("state_name") == "Andaman and Nicobar", "Andaman and Nicobar Islands")
    .when(col("state_name") == "Chattisgarh", "Chhattisgarh")
    .when(col("state_name") == "NCT of Delhi", "Delhi")
    .when(col("state_name") == "Pondicherry", "Puducherry")
    .when(col("state_name") == "Uttrakhand", "Uttarakhand")
    .otherwise(col("state_name")) # Keep original if no match
)
# ---------------------------

# 4. Aggregate: Calculate Price & Volatility per State
# Note: We now group by 'state_clean' instead of 'state_name'
df_market_gold = df_normalized.groupBy("state_clean", "crop_name").agg(
    round(avg("modal_price_rs_quintal"), 2).alias("avg_price"),
    max("modal_price_rs_quintal").alias("max_price_potential"),
    round(expr("coalesce(stddev(modal_price_rs_quintal), 0)"), 2).alias("price_std_dev")
)

# 5. Rename column back to 'state_name' for consistency
df_market_gold = df_market_gold.withColumnRenamed("state_clean", "state_name")

# 6. Feature Engineering: Stability Score
df_market_metrics = df_market_gold.withColumn("volatility_index", 
                                              round(col("price_std_dev") / col("avg_price"), 3))

# 7. Save to Gold (Overwriting the old one)
df_market_metrics.write.format("delta").mode("overwrite").saveAsTable("agriculture.gold.market_financials")

print("Gold Market Financials Table Updated with State Fixes.")
display(df_market_metrics.orderBy("state_name", "crop_name"))

Gold Market Financials Table Updated with State Fixes.


state_name,crop_name,avg_price,max_price_potential,price_std_dev,volatility_index
Andaman and Nicobar Islands,banana,5331.63,8000.0,1364.01,0.256
Andaman and Nicobar Islands,coconut,16.17,20.0,3.19,0.197
Andaman and Nicobar Islands,papaya,3960.0,7000.0,2008.23,0.507
Andhra Pradesh,cotton,8119.88,12389.0,1363.5,0.168
Andhra Pradesh,maize,1805.61,2300.0,251.94,0.14
Andhra Pradesh,mango,2679.92,38000.0,3300.47,1.232
Assam,apple,9100.0,10000.0,699.21,0.077
Bihar,apple,7723.16,24500.0,2690.34,0.348
Bihar,banana,2961.11,8000.0,1232.96,0.416
Bihar,chickpea,7365.47,7600.0,221.97,0.03


In [0]:
%sql
SELECT DISTINCT m.state_name as Market_State
FROM agriculture.gold.market_financials m
LEFT ANTI JOIN agriculture.gold.regional_yield_stats y
ON lower(trim(m.state_name)) = lower(trim(y.state))
ORDER BY Market_State

Market_State
Delhi
Manipur


In [0]:
%sql
-- FINAL RECOMMENDER LOGIC
SELECT 
    y.state,
    y.crop,
    -- 1. The Core Metrics
    y.avg_yield as Yield_Tonnes_Ha,
    m.avg_price as Price_Rs_Quintal,
    m.volatility_index as Risk_Score, -- Lower is better
    
    -- 2. The Financial Calculation (1 Tonne = 10 Quintals)
    round((y.avg_yield * 10 * m.avg_price), 0) as Est_Revenue_Rupees_Ha,
    
    -- 3. The "Smart Score" (Revenue penalized by Risk)
    -- If volatility is high (0.5), we reduce score. If stable (0.1), we keep most value.
    round((y.avg_yield * 10 * m.avg_price) * (1 - m.volatility_index), 0) as Stability_Adjusted_Score

FROM agriculture.gold.regional_yield_stats y
JOIN agriculture.gold.market_financials m
    ON lower(trim(y.state)) = lower(trim(m.state_name)) 
    AND lower(trim(y.crop)) = lower(trim(m.crop_name))

-- Filter for a test state to verify
WHERE y.state = 'Punjab'
ORDER BY Stability_Adjusted_Score DESC

state,crop,Yield_Tonnes_Ha,Price_Rs_Quintal,Risk_Score,Est_Revenue_Rupees_Ha,Stability_Adjusted_Score
Punjab,cotton,3.0,6849.9,0.2,205497.0,164398.0
Punjab,maize,3.19,1408.62,0.277,44935.0,32488.0
Punjab,chickpea,0.98,2050.0,0.0,20090.0,20090.0
Punjab,lentil,0.7,800.0,0.0,5600.0,5600.0
